# Konlpy 영구 저장 경로 설정

In [ ]:
# 패키지 영구 저장
import os, sys
# 구글 마운트 
from google.colab import drive
drive.mount('/content/drive')

# 설치 경로 
# os.mkdir('packages')           # 폴더 생성
my_path = '/content/packages'

# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

# !pip install --target=$my_path konlpy (설치하지 말 것)

Mounted at /content/drive


In [ ]:
!apt -qq -y install fonts-nanum # 폰트 설치
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd 
import re
import pickle
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud

okt = Okt()

file_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/'

# 데이터 불러오기

# 데이터 불러오기

In [ ]:
talk_df = pd.read_csv(file_path + 'TalksetTrain.txt', delimiter='\t', header=None,
                      names=['문장', '분류', '비윤리 여부', '비윤리 강도 평균', '비윤리 강도 총합'])
talk_df

,문장,분류,비윤리 여부,비윤리 강도 평균,비윤리 강도 총합
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,['CENSURE'],True,2.0,10
1,맞아. 사람들이 진짜 개념이없다,['CENSURE'],True,2.0,10
2,인방 보는 남자는 거르는게 맞다,['CENSURE'],True,1.0,5
3,특히 벗방보는 애들은 진짜 거세 시켜야함,"['HATE', 'CENSURE', 'SEXUAL']",True,2.0,10
4,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"['HATE', 'CENSURE']",True,2.0,10
...,...,...,...,...,...
236201,그것보다 어떻게 밥그릇 챙기면서 살 수 있을지 고민하는 게 훨씬 이득이지 않아?,['CENSURE'],True,1.6,8
236202,저 나이 쳐먹고는 아이돌 하겠다고 나대는거임? 세상이 잘 돌아간다,['CENSURE'],True,1.6,8
236203,저런 애들을 뽑아주기라도 하겠음? 그냥 실패의 맛 보러 나온거 아님? ㅋ,['CENSURE'],True,1.6,8
236204,저 여자는 오늘도 레깅스 입네,['IMMORAL_NONE'],False,0.0,0


# 데이터 탐색하기

In [ ]:
# 비식별화 문장 찾기
p = re.compile('#([^#].*?)#')

de_ident_sent = talk_df['문장'].apply(lambda x: p.findall(x))
de_ident_df = talk_df.loc[de_ident_sent.apply(len) > 0,:]
de_ident_df

,문장,분류,비윤리 여부,비윤리 강도 평균,비윤리 강도 총합
12,#@인간및인간집단.인물.정치인.남성#이 이제 급하긴 한가보네,['IMMORAL_NONE'],False,0.0,0
18,"이번에 #@# 유뷰트 봤는데, 3차선에서 역주행하는 차가 있데",['IMMORAL_NONE'],False,0.0,0
21,#@인간및인간집단.인물.정치인.남성# 까면 틀딱 각이다. ㅋㅋㅋ,['CENSURE'],True,1.0,5
22,#@#라고 다 같은 #@#가 아니야,['IMMORAL_NONE'],False,0.0,0
28,#@인간및인간집단.인물.정치인.남성#이 당선 될듯,['IMMORAL_NONE'],False,0.0,0
...,...,...,...,...,...
236044,지금은 과도기라서 그렇지 #@인간및인간집단.인물.정치인.남성#의 부동산 규제는 반드...,['IMMORAL_NONE'],False,0.0,0
236076,"빨갱이도 아니고, #@# 문학 같은 건 대체 왜 연구하는 거냐?","['CENSURE', 'DISCRIMINATION']",True,1.0,5
236118,#@인간및인간집단.인물.정치인.남성# 또 멍청이같은소리하고 다니네,['CENSURE'],True,1.6,8
236158,#@# #@#은 #@#한테서 벗어날 의지가 있는 거냐?,['IMMORAL_NONE'],False,0.0,0


# 데이터 전처리

In [ ]:
# 비식별 문장 제거
talk_df = talk_df[~talk_df['문장'].str.contains(r'#*#')]
talk_df = talk_df.reset_index()
talk_df.drop(columns=['index', '비윤리 여부', '비윤리 강도 평균'], inplace=True)
talk_df

,문장,분류,비윤리 강도 총합
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,['CENSURE'],10
1,맞아. 사람들이 진짜 개념이없다,['CENSURE'],10
2,인방 보는 남자는 거르는게 맞다,['CENSURE'],5
3,특히 벗방보는 애들은 진짜 거세 시켜야함,"['HATE', 'CENSURE', 'SEXUAL']",10
4,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"['HATE', 'CENSURE']",10
...,...,...,...
222240,그것보다 어떻게 밥그릇 챙기면서 살 수 있을지 고민하는 게 훨씬 이득이지 않아?,['CENSURE'],8
222241,저 나이 쳐먹고는 아이돌 하겠다고 나대는거임? 세상이 잘 돌아간다,['CENSURE'],8
222242,저런 애들을 뽑아주기라도 하겠음? 그냥 실패의 맛 보러 나온거 아님? ㅋ,['CENSURE'],8
222243,저 여자는 오늘도 레깅스 입네,['IMMORAL_NONE'],0


In [ ]:
sentences = talk_df['문장']

print(f'TalkSet 데이터에서 문장은 {len(sentences)}개 입니다.')

TalkSet 데이터에서 문장은 222245개 입니다.


In [ ]:
def preprocessing(review, BanList):
    # 한글, 영문자, 숫자 등의 문자만 추출합니다.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ0-9a-zA-Z\s]", "", review)    
    
    # okt 형태소 분석을 합니다.
    T = okt.pos(review_text, stem=True, norm=True)

    # 유의미한 형태소만을 추출합니다. (조사 제외)
    PosList = []
    for tuples in T:
        if tuples[0] not in BanList and tuples[1] not in ['Josa']:          
            PosList.append(tuples)
            
    # 분석에 필요한 형태소의 모임만을 return 합니다.
    return PosList

In [ ]:
sent_lst = talk_df['문장']
pos_lst = []
BanList = []

print('Start!', end=' ')
cnt = 0
for seq in sent_lst:
    seq_pos = preprocessing(seq, BanList)
    pos_lst.append(seq_pos)
    # print(seq_morphs)

    # 메세지 출력
    cnt += 1
    if cnt % 5000 == 0:
        print(f'> {cnt}', end=' ')
    if cnt % 50000 == 0:
        print()
print('End!')

Start! > 5000 > 10000 > 15000 > 20000 > 25000 > 30000 > 35000 > 40000 > 45000 > 50000 
> 55000 > 60000 > 65000 > 70000 > 75000 > 80000 > 85000 > 90000 > 95000 > 100000 
> 105000 > 110000 > 115000 > 120000 > 125000 > 130000 > 135000 > 140000 > 145000 > 150000 
> 155000 > 160000 > 165000 > 170000 > 175000 > 180000 > 185000 > 190000 > 195000 > 200000 
> 205000 > 210000 > 215000 > 220000 End!


In [ ]:
# 저장하기 (pickle로 저장)
with open(file_path + 'okt_pos.pkl', 'wb') as f:
    pickle.dump(pos_lst, f)